In [2]:
import sys
import subprocess

# Check if PyWavelets is installed, if not, install it
if 'pywt' not in sys.modules:
    print("PyWavelets not found. Installing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "PyWavelets"])
    print("PyWavelets installed successfully.")

import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNet, VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D
import os
from google.colab import drive
from scipy.signal import welch
from scipy.stats import skew, kurtosis
from pywt import wavedec
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)



PyWavelets not found. Installing...
PyWavelets installed successfully.
Mounted at /content/drive


In [3]:
def get_valid_directory():
    while True:
        directory = 'My Drive/Bonn dataset'
        full_path = os.path.join('/content/drive', directory)
        if os.path.isdir(full_path):
            return full_path
        else:
            print(f"The directory '{full_path}' does not exist. Please try again.")

def load_eeg_data(file_path):
    try:
        data = np.loadtxt(file_path)
        print(f"Loaded data from {file_path}, shape: {data.shape}")

        if len(data) != 4097:
            print(f"Unexpected data length in {file_path}. Expected 4097, got {len(data)}")
            return None

        padded_data = np.pad(data, (0, 4225 - 4097), 'constant')
        data_2d = padded_data.reshape(1, 65, 65, 1)
        return data_2d, data  # Return both 2D and 1D data
    except Exception as e:
        print(f"Error loading file {file_path}: {str(e)}")
        return None, None



In [4]:
def create_lenet():
    model = Sequential([
        Conv2D(6, kernel_size=5, activation='relu', input_shape=(65, 65, 1)),
        AveragePooling2D(pool_size=(2, 2)),
        Conv2D(16, kernel_size=5, activation='relu'),
        AveragePooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(120, activation='relu'),
        Dense(84, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model



In [5]:
def extract_features(data, model):
    try:
        feature_extractor = tf.keras.Model(inputs=model.inputs, outputs=model.layers[-2].output)
        features = feature_extractor.predict(data)
        return features
    except Exception as e:
        print(f"Error extracting features: {str(e)}")
        return None

def extract_statistical_features(signal):
    return [
        np.mean(signal),
        np.std(signal),
        np.min(signal),
        np.max(signal),
        np.median(signal),
        skew(signal),
        kurtosis(signal),
        np.mean(np.abs(np.diff(signal))),  # Mean absolute difference
        np.sum(np.abs(signal) ** 2) / len(signal),  # Energy
        np.sum(np.abs(signal)) / len(signal),  # Average amplitude
    ]

def extract_frequency_features(signal, fs=173.61):
    f, psd = welch(signal, fs, nperseg=len(signal)//2)
    total_power = np.sum(psd)
    return [
        np.sum(psd[(f >= 0.5) & (f <= 4)]) / total_power,     # Delta band power (relative)
        np.sum(psd[(f > 4) & (f <= 8)]) / total_power,        # Theta band power (relative)
        np.sum(psd[(f > 8) & (f <= 13)]) / total_power,       # Alpha band power (relative)
        np.sum(psd[(f > 13) & (f <= 30)]) / total_power,      # Beta band power (relative)
        np.sum(psd[(f > 30) & (f <= 100)]) / total_power,     # Gamma band power (relative)
        np.mean(f * psd) / total_power,  # Spectral centroid
        np.sqrt(np.sum(f**2 * psd) / np.sum(psd)),  # Spectral spread
        -np.sum(psd * np.log2(psd + 1e-10)) / np.log2(len(psd)),  # Spectral entropy
    ]

def extract_wavelet_features(signal, wavelet='db4', level=5):
    coeffs = wavedec(signal, wavelet, level=level)
    return [np.mean(np.abs(c)) for c in coeffs] + [np.std(c) for c in coeffs]



In [6]:
def process_file(file_path, mobilenet, vgg16, lenet, category, save_dir):
    try:
        print(f"Processing file: {file_path}")
        print(f"Category: {category}")
        print(f"Save directory: {save_dir}")

        eeg_data_2d, eeg_data_1d = load_eeg_data(file_path)
        if eeg_data_2d is not None and eeg_data_1d is not None:
            # Preprocess data for MobileNet and VGG16
            eeg_data_rgb = np.repeat(eeg_data_2d, 3, axis=-1)  # Convert to RGB
            eeg_data_rgb = tf.image.resize(eeg_data_rgb, (224, 224))  # Resize to 224x224

            # Extract features
            mobilenet_features = extract_features(eeg_data_rgb, mobilenet)
            vgg16_features = extract_features(eeg_data_rgb, vgg16)
            lenet_features = extract_features(eeg_data_2d, lenet)

            # Extract additional features
            statistical_features = extract_statistical_features(eeg_data_1d)
            frequency_features = extract_frequency_features(eeg_data_1d)
            wavelet_features = extract_wavelet_features(eeg_data_1d)

            if all([mobilenet_features is not None, vgg16_features is not None, lenet_features is not None]):
                # Combine all features
                all_features = np.concatenate([
                    mobilenet_features.flatten(),
                    vgg16_features.flatten(),
                    lenet_features.flatten(),
                    statistical_features,
                    frequency_features,
                    wavelet_features
                ])

                # Save the features to a .npz file
                file_name = os.path.basename(file_path).replace('.txt', '.npz')
                save_path = os.path.join(save_dir, file_name)
                np.savez(save_path, features=all_features, category=category)
                print(f"Saved features to {save_path}")
                print(f"Total number of features: {len(all_features)}")
            else:
                print(f"Failed to extract features for {file_path}")
        else:
            print(f"Failed to load data from {file_path}")

    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
    finally:
        tf.keras.backend.clear_session()  # Clear session to free up memory



In [7]:
def main():
    try:
        # Get the correct directory path from the user
        data_dir = get_valid_directory()

        # Create folders to save the processed data
        save_dir_base = '/content/drive/MyDrive/Bonn_processed5'
        save_dir_normal = os.path.join(save_dir_base, 'Normal')
        save_dir_interictal = os.path.join(save_dir_base, 'Interictal')
        save_dir_ictal = os.path.join(save_dir_base, 'Ictal')
        os.makedirs(save_dir_normal, exist_ok=True)
        os.makedirs(save_dir_interictal, exist_ok=True)
        os.makedirs(save_dir_ictal, exist_ok=True)

        # Load pre-trained models
        mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        lenet = create_lenet()

        # Map folder names to categories
        category_map = {
            'Z(a)': 'Normal',
            'O(b)': 'Normal',
            'N(c)': 'Interictal',
            'F(d)': 'Ictal',
            'S(e)': 'Ictal'
        }

        # Walk through the dataset directories
        for root, dirs, files in os.walk(data_dir):
            folder_name = os.path.basename(root)
            category = category_map.get(folder_name)

            if category:
                # Set save directory based on the category
                if category == 'Normal':
                    save_dir = save_dir_normal
                elif category == 'Interictal':
                    save_dir = save_dir_interictal
                elif category == 'Ictal':
                    save_dir = save_dir_ictal
                else:
                    print(f"Unknown category: {category}")
                    continue

                print(f"Processing folder: {folder_name} (Category: {category})")

                for file in files:
                    file_path = os.path.join(root, file)
                    # Make file extension check case-insensitive
                    if os.path.exists(file_path) and file.lower().endswith('.txt'):
                        print(f"Processing file: {file_path}")
                        process_file(file_path, mobilenet, vgg16, lenet, category, save_dir)
                    else:
                        print(f"File {file_path} does not exist or is not accessible.")
            else:
                print(f"Skipping folder: {folder_name} (No matching category)")

    except Exception as e:
        print(f"An error occurred in the main function: {str(e)}")



if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Skipping folder: Bonn dataset (No matching category)
Processing folder: N(c) (Category: Interictal)
Processing file: /content/drive/My Drive/Bonn dataset/N(c)/N093.TXT
Processing file: /content/drive/My Drive/Bonn dataset/N(c)/N093.TXT
Category: Interictal
Save directory: /content/drive/MyDrive/Bonn_processed5/Interictal
Loaded data from /content/drive/My Drive/Bonn dataset/N(c)/N093.TXT, shape: (4097,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
Saved features to /content/drive/MyDrive/Bonn_processed5/Interictal/N093.TXT
Total number of features: 150642
Processing file: /content/drive/My Drive/Bonn dataset/N(c)/N087.TXT
Processing file: /content/drive/My Drive/Bonn dataset/N(c)/N087.TXT
Category: Interictal
Save directory: /content/drive/MyDrive/Bonn_processed5/Interictal
Loaded data from /content/drive/My Drive/Bonn dataset/N(c)/N087.TXT, shape: (4097,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 412ms/step
Saved features to /content/drive/MyDrive/Bonn_processed5/Interictal/N087.TXT
Total number of features: 150642
Processing file: /content/drive/My Drive/Bonn dataset/N(c)/N095.TXT
Processing file: /content/drive/My Drive/Bonn dataset/N(c)/N095.TXT
Category: Interictal
Save directory: /content/drive/MyDrive/Bonn_processed5/Interictal
Loaded data from /content/drive/My Drive/Bonn dataset/N(c)/N095.TXT, shape: (4097,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step
Saved features to /content/drive/MyDrive/Bonn_processed5/Interictal/N095.TXT
Total number of features: 150642
Processing file: /content/drive/My Drive/Bonn dataset/N(c)/N001.TXT
Processing file: /content/drive/My Drive/Bonn dataset/N(c)/N001.TXT
Category: Interictal
Save directory: /content/drive/MyDrive/Bonn_processed5/Interictal
Loaded data from /content/drive/My Drive/Bonn dataset/N(c)/N001.TXT, shape: (4097,)
1/1 ━━━━━━━━━━━━